In [2]:
import pandas as pd
import seaborn as sns

%load_ext google.cloud.bigquery

# Qualify

In [16]:
%%bigquery --project nbcu-ds-sandbox-a-001

SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
        , current_date() -- disable results caching
FROM       
        (
            SELECT adobe_tracking_id
                    , report_date
                    , paying_account_flag as paying_account_flag_today
                    , LAG(paying_account_flag,1) OVER ( partition by adobe_tracking_id order by report_date  ) as paying_account_flag_yestd -- paying flag yesterday
            FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
            WHERE report_date between '2022-01-01' and '2022-01-07' -- don't add report_date since we need to distinguish upgrades from beginnning of time 
        )
WHERE paying_account_flag_today = 'Paying' AND paying_account_flag_yestd = 'NonPaying'
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number,f0_
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,1,2023-03-03
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,1,2023-03-03
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,1,2023-03-03
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,1,2023-03-03
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,1,2023-03-03
...,...,...,...,...
232286,GLASehsS05Pc8PMPNCtfU6jgQzaS7AoFFADyzqYHB+M=,2022-01-07,2,2023-03-03
232287,eXcU8h+m46HjlaJVUz/n4u005866yleewMshl8heX4M=,2022-01-07,2,2023-03-03
232288,kT2X7Ykx4m9Pry/Z3QsIz4omu0u06VNUGWRORrs5z9s=,2022-01-07,2,2023-03-03
232289,uRoiS67Kr6o55445yaIQAUlIq8v0iWPG0BtNcR4CvU8=,2022-01-07,2,2023-03-03


In [17]:
%%bigquery --project nbcu-ds-sandbox-a-001

with cte as (
SELECT adobe_tracking_id
        , report_date
        , current_date() -- disable results caching
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
WHERE report_date between '2022-01-01' and '2022-01-07'
QUALIFY (paying_account_flag = 'Paying') and (LAG(paying_account_flag,1) over(partition by adobe_tracking_id order by report_date) = 'NonPaying')
)
SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
FROM cte
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,1
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,1
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,1
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,1
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,1
...,...,...,...
232286,GLASehsS05Pc8PMPNCtfU6jgQzaS7AoFFADyzqYHB+M=,2022-01-07,2
232287,eXcU8h+m46HjlaJVUz/n4u005866yleewMshl8heX4M=,2022-01-07,2
232288,kT2X7Ykx4m9Pry/Z3QsIz4omu0u06VNUGWRORrs5z9s=,2022-01-07,2
232289,uRoiS67Kr6o55445yaIQAUlIq8v0iWPG0BtNcR4CvU8=,2022-01-07,2


In [15]:
%%bigquery --project nbcu-ds-sandbox-a-001

SELECT adobe_tracking_id
        , report_date
        , row_number() OVER(partition by adobe_tracking_id order by report_date ) as upgrade_row_number -- rank the number of times a user upgrade
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
WHERE report_date between '2022-01-01' and '2022-01-07'
QUALIFY (paying_account_flag = 'Paying') and (LAG(paying_account_flag,1) over(partition by adobe_tracking_id order by report_date) = 'NonPaying')
order by 3,2,1

Query is running:   0%|          |

Downloading:   0%|          |

,adobe_tracking_id,report_date,upgrade_row_number
0,++9P2+WhbQt4rjQWiakE+OtWVZ5gBDESVZlhsFLO/o8=,2022-01-02,2
1,++BZuALDJvn4Uxxo6fv37L4JkSqKDdsnfBGUu3rOhGc=,2022-01-02,2
2,++O4U6CfhQofob0caUsUAiWrmBLg/1V1xn7K859sX6Q=,2022-01-02,2
3,++aIiiCONAa+hayeE4idJI4qgLMak1RT9M76RdEMu+4=,2022-01-02,2
4,++bwvsqrgOQEDtjMqCdFtQo0QWbRRZgNzBJRTeDJRYI=,2022-01-02,2
...,...,...,...
232286,zytjwALhwovNtYv1jrNBivgYnpB0P8x4lXqbE0Jbr24=,2022-01-07,7
232287,zz6VfVUYUU048zOqEiQ6okOuDJ07Rv5MVI1hVgWHjqI=,2022-01-07,7
232288,zz7kz8DR4vbk37Z6qzCdA/3gJ+2ld66sEEPPfX0fkSA=,2022-01-07,7
232289,zzQ7pCcZjrj1HYboqlJIGWHjjOJLfufzzA8oeJPXXqU=,2022-01-07,7


# Groupby investigation

In [88]:
# Equivalent to declare
# import params in %%bigquery with --params $params
params = {
    'report_start_date': '2023-01-01',
    'report_end_date': '2023-01-07'
}

### Original

In [90]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted` AS ( -- everyone who have received emails in the month, and opened at least one email in the past
    SELECT DISTINCT adobe_tracking_id 
    FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE` 
    WHERE event_name = 'Email Deliveries' 
    AND event_date BETWEEN @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [91]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout` AS ( 
    SELECT  distinct holdout.adobe_tracking_id
    FROM
    (
        SELECT  distinct TrackingId AS adobe_tracking_id -- distinct Hold_Out_Type_Current --cohort, count(distinct TrackingId)
        FROM `nbcu-ds-prod-001.PeacockDataMartMarketingGold.HOLDOUT_GROUP`
        WHERE cohort = 'January2023'
        AND Hold_Out_Type_Current = 'Owned Email Holdout'
        -- Exclude those who are assigned to Email Holdout but actually received emails 
    ) holdout
    LEFT JOIN
    (
        SELECT  DISTINCT adobe_tracking_id
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
        WHERE event_name = 'Email Deliveries'
        AND event_date >= @report_start_date
        AND event_date <= @report_end_date 
    ) delivered -- Email Delivered ever
    ON holdout.adobe_tracking_id = delivered.adobe_tracking_id AND delivered.adobe_tracking_id is null
)

Query is running:   0%|          |

""


### Account type tables

In [92]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_no_account_type` AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.adobe_tracking_id as aid
            , cohort
    FROM         -- Include email engagers only: users who have at least 1 'Other Opens' in entire user histoy; exclude Oct 2022 Privacy email
     (SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Opens' and machineOpen is null
            AND date(eventTimestamp) <= @report_end_date
            AND campaignName NOT IN ('TransactionalTermsOfUseEngaged20221028', 'TransactionalTermsOfUseNONEngaged20221028') -- Exclude email openers of Oct privacy email
        ) Email_Engagers --opened email at least once, using other open 

          INNER JOIN `nbcu-ds-sandbox-a-001.sl_sandbox.Braze_Id_Adobe_Id_Map` mapping
          ON mapping.bid = Email_Engagers.identity

          INNER JOIN
              (SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted`
                  UNION ALL
                  SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout`
              ) a
          ON a.adobe_tracking_id = mapping.aid
        
        -- for after 2021/july, email channel only, take out all abandon MAAs
        INNER JOIN 
        (SELECT DISTINCT adobe_tracking_id FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
            WHERE report_date = @report_end_date
            AND date_of_last_view IS NOT NULL 
        ) abandon_maa 
        ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id
)

Query is running:   0%|          |

""


In [93]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts` AS (
    -- email targetable and holdout cohort, users signed up after 2020/8/11
     SELECT distinct a.adobe_tracking_id as aid
            , cohort
            , user.account_type
    FROM         -- Include email engagers only: users who have at least 1 'Other Opens' in entire user histoy; exclude Oct 2022 Privacy email
     (SELECT DISTINCT identity 
            FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
            WHERE eventName = 'Email Opens' and machineOpen is null
            AND date(eventTimestamp) <= @report_end_date
            AND campaignName NOT IN ('TransactionalTermsOfUseEngaged20221028', 'TransactionalTermsOfUseNONEngaged20221028') -- Exclude email openers of Oct privacy email
        ) Email_Engagers --opened email at least once, using other open 

          INNER JOIN `nbcu-ds-sandbox-a-001.sl_sandbox.Braze_Id_Adobe_Id_Map` mapping
          ON mapping.bid = Email_Engagers.identity

          INNER JOIN
              (SELECT *, 'Email_Targeted' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Targeted`
                  UNION ALL
                  SELECT *, 'Holdout' as cohort from `nbcu-ds-sandbox-a-001.SLi_sandbox.Holdout`
              ) a
          ON a.adobe_tracking_id = mapping.aid
              
          --add attribute: account_type 
          INNER JOIN ( SELECT * FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` WHERE report_date = @report_end_date ) user
          ON a.adobe_tracking_id = user.adobe_tracking_id 
        

        -- for after 2021/july, email channel only, take out all abandon MAAs
        INNER JOIN 
        (SELECT DISTINCT adobe_tracking_id FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
            WHERE report_date = @report_end_date
            AND date_of_last_view IS NOT NULL 
        ) abandon_maa 
        ON a.adobe_tracking_id = abandon_maa.adobe_tracking_id
)


Query is running:   0%|          |

""


In [95]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts_no_account_type`

Query is running:   0%|          |

Downloading:   0%|          |

,aid,cohort
0,iCNU3FVvdky2wJTh+o+rhCidJutDL5z5fH+tX7rH8sg=,Email_Targeted
1,vPhjIqVoUGlcCixejrR5ka3h6OLFRcbI78oBycrT6nQ=,Email_Targeted
2,rZqJPH/boGpPK45Mzkr6G5WC/TKZb0VLYN/o12bfq30=,Email_Targeted
3,mTj7alfHvX4+LXk1MdhTBOcP7QwM6DRRXLKW8W7WL+E=,Email_Targeted
4,R1Z+P9q8JhxXck/srnZn8IcYaVvcBpiX8AQVD6+oR2I=,Email_Targeted
...,...,...
26685333,uWFlXgV/TDrgXpOteDqZgUd9xipqY+AWWnWOVRVhDAo=,Email_Targeted
26685334,ZLpJanxUKmXvLjiXyXyNWk0pwIDsrH8qJlspGnSNwJs=,Email_Targeted
26685335,WIdWPFNwEBucWNTYzPvxcbdGpm+8z6Ra9BWtoSGUuW4=,Email_Targeted
26685336,PBF0KhC9xlEAHMXrFl/orXLBb5EeRg4dnZ/Ex0w+nFY=,Email_Targeted


In [94]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts`

Query is running:   0%|          |

Downloading:   0%|          |

,aid,cohort,account_type
0,T3576lZJ6PBAHt4kfKJmLJPj6R+aQCZB0/J9VxJdZA4=,Email_Targeted,Free
1,bRtASLDM2MPJ5DJtxBH/kPXW9hygzFXVc1CrdU4mxdE=,Email_Targeted,Free
2,4hfPZRv2n+idvj2oH9X8UNUi9oOyxbqwuqi9HAn/gNs=,Email_Targeted,Free
3,Zzi/rgcXmW/6k1T2U4QVyjMsMjgDnzOzw68/dt334so=,Email_Targeted,Free
4,GwRixl2MgdGJCKvql3AjT5a14Vb7eDJRS/WsTcnySxI=,Email_Targeted,Free
...,...,...,...
26685333,HcoD6rP4xr2Skwt78xbd2i17TOljiPAbVBQ9uOJvt4U=,Email_Targeted,Paying SVOD
26685334,FuuowNrrLGimofSlaxtOLuU0TOve1QLvAZO+puAIjnU=,Email_Targeted,Paying SVOD
26685335,TikJqd6JyP+KNnusvjb7YcpuztAK/872lh+joTRDnwY=,Email_Targeted,Paying SVOD
26685336,Zdud7UDy7DgvBeAe2l/AKrc+4oJoi17fxe6/rs6JZns=,Email_Targeted,Paying SVOD


In [96]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing` AS (
    SELECT 
        adobe_tracking_id
        , COUNT (DISTINCT CASE WHEN VIDEO.num_views_started =1 THEN video.adobe_tracking_id ELSE NULL END) AS Distinct_Content_Starts
        , SUM (VIDEO.num_views_started ) AS Total_Content_Starts
        , SUM(VIDEO.num_seconds_played_no_ads)/3600 AS Viewing_Time
        , COUNT(DISTINCT session_id) AS Distinct_Viewing_Sessions 
        , COUNT(DISTINCT(CASE WHEN (num_seconds_played_no_ads > CASE WHEN lower(consumption_type) = 'virtual channel' THEN 299 ELSE 0 END)
                and (num_views_started>0) THEN CASE WHEN (lower(consumption_type) = "shortform") THEN "Shortform"
                                                    WHEN lower(franchise) != 'other' THEN franchise ELSE display_name
                                                    END
                end)) as Repertoire_Pavo_Method
                
    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts` a

    INNER JOIN  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO
        ON VIDEO.adobe_tracking_id = a.aid
        AND adobe_date between @report_start_date AND @report_end_date
    GROUP BY 1
)


Query is running:   0%|          |

""


In [97]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom` AS ( --'Lapsing_Users'
    SELECT distinct adobe_tracking_id, date_of_last_view -- no need for max (days_since_last_view), can be saved multiple times, dedup later
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
    WHERE report_date BETWEEN @report_start_date AND @report_end_date 
    AND days_since_last_view BETWEEN 15 AND 29 -- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
)

Query is running:   0%|          |

""


In [98]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Num` as (
        SELECT distinct a.adobe_tracking_id      
        FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom` a
        INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO` VIDEO 
            ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
            AND adobe_date >= @report_start_date AND adobe_date<= @report_end_date
            AND VIDEO.adobe_date >= DATE_ADD(date_of_last_view, INTERVAL 15 day)
            AND VIDEO.adobe_date <= DATE_ADD(date_of_last_view, INTERVAL 29 day)
)

Query is running:   0%|          |

""


In [99]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom` AS ( --'Lapsed_Users'
        SELECT distinct adobe_tracking_id, date_of_last_view
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_PRIMARY_DEVICES` 
        WHERE report_date BETWEEN @report_start_date AND @report_end_date 
        AND days_since_last_view >=30 AND days_since_last_view <= 90-- this guarantees we are only getting people who have at least past the 'lapsing' phase in the time period.
    )

Query is running:   0%|          |

""


In [100]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Num` AS (
    SELECT distinct a.adobe_tracking_id

    FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom` a
    INNER JOIN `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_VIDEO`  VIDEO 
        ON a.adobe_tracking_id = VIDEO.adobe_tracking_id 
        AND adobe_date >= @report_start_date AND adobe_date<= @report_end_date
        AND VIDEO.adobe_date >= DATE_ADD(date_of_last_view, INTERVAL 30 day)
    )

Query is running:   0%|          |

""


In [101]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Denom` AS (
-- upgrade metric 2.0: nonpaying to paying
    SELECT distinct adobe_tracking_id 
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
    WHERE paying_account_flag = 'NonPaying' 
    AND USER.report_date BETWEEN @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [102]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Num` AS (
-- find all users who have upgraded at least once in the month of
    SELECT  distinct adobe_tracking_id
    FROM
        (SELECT  
                report_date
             , adobe_tracking_id     
         FROM  `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`  USER
         WHERE entitlement_change_flag IN ('Upgrade: Free to Premium'
                                                    , 'Upgrade: Free to Premium+'
                                                    , 'Upgrade: Premium to Premium+') 
         AND paying_account_flag = 'Paying'                                            
            AND USER.report_date BETWEEN @report_start_date and @report_end_date
        ) 
)  

--this doesn't come from upgrade_denom, maybe it should be?

Query is running:   0%|          |

""


In [103]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Churn_Denom` AS (
    SELECT distinct adobe_tracking_id
    FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER` 
    WHERE paying_account_flag = 'Paying'
    AND auto_renew_flag = 'OFF'
    AND report_date BETWEEN @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [104]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Churn_Num` AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
            ,auto_renew_flag                                                                      AS auto_renew_flag_today
            ,LEAD(auto_renew_flag,1) OVER ( partition by adobe_tracking_id ORDER BY report_date ) AS auto_renew_flag_next_day
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE report_date BETWEEN @report_start_date AND @report_end_date
        ORDER BY 1, 2 
    )
    WHERE auto_renew_flag_today = 'OFF'
    AND auto_renew_flag_next_day = 'ON'
)

Query is running:   0%|          |

""


In [105]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Denom` AS (
    select distinct adobe_tracking_id
    from `nbcu-ds-sandbox-a-001.sl_sandbox.upgrade_never`
    where report_date between @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [106]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Num` AS (
    SELECT  distinct adobe_tracking_id
    FROM `nbcu-ds-sandbox-a-001.sl_sandbox.upgrade_date_rank`
    WHERE upgrade_row_number = 1
    AND report_date BETWEEN @report_start_date AND @report_end_date
)

Query is running:   0%|          |

""


In [107]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Denom` AS (
    SELECT  distinct adobe_tracking_id
    FROM
    (
        SELECT  adobe_tracking_id
            ,report_date
        FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_USER`
        WHERE paying_account_flag = 'NonPaying'
        AND report_date BETWEEN @report_start_date AND @report_end_date 
    )
    WHERE adobe_tracking_id NOT IN ( SELECT distinct adobe_tracking_id FROM `nbcu-ds-sandbox-a-001.sl_sandbox.upgrade_never` WHERE report_date BETWEEN @report_start_date AND @report_end_date )
)

Query is running:   0%|          |

""


In [108]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Num` AS (

    select distinct adobe_tracking_id
    from `nbcu-ds-sandbox-a-001.sl_sandbox.upgrade_date_rank`
    where upgrade_row_number > 1
    and report_date between @report_start_date and @report_end_date
)

Query is running:   0%|          |

""


In [109]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Denom` AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where date = @report_end_date
  and entitlement = 'Paid'
)


Query is running:   0%|          |

""


In [110]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Num` AS (
  select adobe_tracking_id
  from `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_CHURN`
  where date = @report_end_date
  and entitlement = 'Paid'
  and Churn_flag = 'Churn'
)

Query is running:   0%|          |

""


## Put everything together

In [111]:
%%bigquery --project nbcu-ds-sandbox-a-001 --params $params
CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.CTE_1` AS (
  SELECT 
    @report_start_date AS Report_Month
        , a.Account_Type
    
        , count(distinct case when cohort = 'Email_Targeted' then a.aid end ) as Distinct_Cohort_Size_Targeted
        , count(distinct case when cohort = 'Holdout' then a.aid end) as Distinct_Cohort_Size_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then video.adobe_tracking_id end ) as Total_Returns_Targeted
        , count(distinct case when cohort = 'Holdout' then video.adobe_tracking_id end) as Total_Returns_Holdout

        , sum(case when cohort = 'Email_Targeted' then video.Viewing_Time end ) as Total_Usage_Targeted
        , sum(case when cohort = 'Holdout' then video.Viewing_Time end) as Total_Usage_Holdout

        , sum(case when cohort = 'Email_Targeted' then video.Repertoire_Pavo_Method end ) as Total_Repertoire_Targeted
        , sum(case when cohort = 'Holdout' then video.Repertoire_Pavo_Method end) as Total_Repertoire_Holdout

        , sum(case when cohort = 'Email_Targeted' then video.Distinct_Viewing_Sessions end ) as Total_Viewing_Sessions_Targeted
        , sum(case when cohort = 'Holdout' then video.Distinct_Viewing_Sessions end) as Total_Viewing_Sessions_Holdout


        , count(distinct case when cohort = 'Email_Targeted' then Winback_Denom.adobe_tracking_id end ) as Winback_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then Winback_Num.adobe_tracking_id end) as Winback_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then Winback_Denom.adobe_tracking_id end ) as Winback_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then Winback_Num.adobe_tracking_id end) as Winback_Num_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then Save_Denom.adobe_tracking_id end ) as Save_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then Save_Num.adobe_tracking_id end) as Save_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then Save_Denom.adobe_tracking_id end ) as Save_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then Save_Num.adobe_tracking_id end) as Save_Num_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Denom.adobe_tracking_id end ) as Upgrades_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then Upgrade_Num.adobe_tracking_id end) as Upgrades_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then Upgrade_Denom.adobe_tracking_id end ) as Upgrades_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then Upgrade_Num.adobe_tracking_id end) as Upgrades_Num_Holdout



        , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Denom.adobe_tracking_id end ) as Total_New_Upgrade_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then New_Upgrade_Num.adobe_tracking_id end) as Total_New_Upgrade_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then New_Upgrade_Denom.adobe_tracking_id end ) as Total_New_Upgrade_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then New_Upgrade_Num.adobe_tracking_id end) as Total_New_Upgrade_Num_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Denom.adobe_tracking_id end ) as Total_Paid_Winbacks_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then Paid_Winbacks_Num.adobe_tracking_id end) as Total_Paid_Winbacks_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Denom.adobe_tracking_id end ) as Total_Paid_Winbacks_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then Paid_Winbacks_Num.adobe_tracking_id end) as Total_Paid_Winbacks_Num_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Denom.adobe_tracking_id end ) as Total_Paid_Churn_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then Paid_Churn_Num.adobe_tracking_id end) as Total_Paid_Churn_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then Paid_Churn_Denom.adobe_tracking_id end ) as Total_Paid_Churn_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then Paid_Churn_Num.adobe_tracking_id end) as Total_Paid_Churn_Num_Holdout

        , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Denom.adobe_tracking_id end ) as EOM_Paid_Churn_Denom_Targeted
        , count(distinct case when cohort = 'Email_Targeted' then EOM_Paid_Churn_Num.adobe_tracking_id end) as EOM_Paid_Churn_Num_Targeted
        , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Denom.adobe_tracking_id end ) as EOM_Paid_Churn_Denom_Holdout
        , count(distinct case when cohort = 'Holdout' then EOM_Paid_Churn_Num.adobe_tracking_id end) as EOM_Paid_Churn_Num_Holdout


  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.All_Cohorts` a
  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Video_Viewing` video
      ON a.aid = video.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Denom` Winback_Denom
      ON a.aid = Winback_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Winback_Num` Winback_Num
      ON Winback_Denom.adobe_tracking_id = Winback_Num.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Denom` Save_Denom
      ON a.aid = Save_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Save_Num` Save_Num
      ON Save_Denom.adobe_tracking_id = Save_Num.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Denom` Upgrade_Denom
      ON a.aid = Upgrade_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Upgrade_Num` Upgrade_Num
      ON Upgrade_Denom.adobe_tracking_id = Upgrade_Num.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Denom` New_Upgrade_Denom
      on a.aid = New_Upgrade_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.New_Upgrade_Num` New_Upgrade_Num
      on New_Upgrade_Denom.adobe_tracking_id = New_Upgrade_Num.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Denom` Paid_Winbacks_Denom
      on a.aid = Paid_Winbacks_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Winbacks_Num` Paid_Winbacks_Num
      on Paid_Winbacks_Num.adobe_tracking_id = Paid_Winbacks_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Churn_Denom` Paid_Churn_Denom
      on a.aid = Paid_Churn_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Paid_Churn_Num` Paid_Churn_Num
      on Paid_Churn_Denom.adobe_tracking_id = Paid_Churn_Num.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Denom` EOM_Paid_Churn_Denom
      on a.aid = EOM_Paid_Churn_Denom.adobe_tracking_id

  LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.EOM_Paid_Churn_Num` EOM_Paid_Churn_Num
      on EOM_Paid_Churn_Denom.adobe_tracking_id = EOM_Paid_Churn_Num.adobe_tracking_id

  Group By 1,2
  )

Query is running:   0%|          |

""


In [112]:
%%bigquery df --project nbcu-ds-sandbox-a-001 --params $params
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.CTE_1`

Query is running:   0%|          |

Downloading:   0%|          |

In [113]:
%%bigquery df2 --project nbcu-ds-sandbox-a-001 --params $params
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.CTE_1_no_account_type`

Query is running:   0%|          |

Downloading:   0%|          |

In [114]:
df

,Report_Month,Account_Type,Distinct_Cohort_Size_Targeted,Distinct_Cohort_Size_Holdout,Total_Returns_Targeted,Total_Returns_Holdout,Total_Usage_Targeted,Total_Usage_Holdout,Total_Repertoire_Targeted,Total_Repertoire_Holdout,...,Total_Paid_Winbacks_Denom_Holdout,Total_Paid_Winbacks_Num_Holdout,Total_Paid_Churn_Denom_Targeted,Total_Paid_Churn_Num_Targeted,Total_Paid_Churn_Denom_Holdout,Total_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Targeted,EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Holdout,EOM_Paid_Churn_Num_Holdout
0,2023-01-01,Bundled Premium,5927808,389923,1950493,117337,1.110446e+07,6.839584e+05,8966069,534070,...,13761,0,560,7,31,0,8986,8986,496,496
1,2023-01-01,Free,11891560,940506,613630,38048,8.575185e+05,5.494802e+04,1641221,104432,...,228845,7,28686,3,1360,0,415836,415836,22684,22684
2,2023-01-01,Paying SVOD,7140051,395490,4354090,240449,3.513716e+07,1.986234e+06,31124570,1725762,...,2886,2182,129298,1411,6674,67,6583527,0,368846,0


In [115]:
df2

,Report_Month,Distinct_Cohort_Size_Targeted,Distinct_Cohort_Size_Holdout,Total_Returns_Targeted,Total_Returns_Holdout,Total_Usage_Targeted,Total_Usage_Holdout,Total_Repertoire_Targeted,Total_Repertoire_Holdout,Total_Viewing_Sessions_Targeted,...,Total_Paid_Winbacks_Denom_Holdout,Total_Paid_Winbacks_Num_Holdout,Total_Paid_Churn_Denom_Targeted,Total_Paid_Churn_Num_Targeted,Total_Paid_Churn_Denom_Holdout,Total_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Targeted,EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Holdout,EOM_Paid_Churn_Num_Holdout
0,2023-01-01,21991729,1724884,3066853,193350,7.234022e+06,458021.487778,8493831,531550,6090173,...,241058,0,96060,0,5931,0,6191726,336609,390716,23053


In [116]:
# By account type
df.sum().drop(['Report_Month', 'Account_Type']).squeeze()

Distinct_Cohort_Size_Targeted                24959419
Distinct_Cohort_Size_Holdout                  1725919
Total_Returns_Targeted                        6918213
Total_Returns_Holdout                          395834
Total_Usage_Targeted                  47099142.309723
Total_Usage_Holdout                    2725140.921667
Total_Repertoire_Targeted                    41731860
Total_Repertoire_Holdout                      2364264
Total_Viewing_Sessions_Targeted              38168514
Total_Viewing_Sessions_Holdout                2172846
Winback_Denom_Targeted                        4368363
Winback_Num_Targeted                           368742
Winback_Denom_Holdout                          262523
Winback_Num_Holdout                             21216
Save_Denom_Targeted                           3114358
Save_Num_Targeted                              473075
Save_Denom_Holdout                             184038
Save_Num_Holdout                                26675
Upgrades_Denom_Targeted     

In [117]:
# total
df2.T.drop(['Report_Month']).squeeze()

Distinct_Cohort_Size_Targeted               21991729
Distinct_Cohort_Size_Holdout                 1724884
Total_Returns_Targeted                       3066853
Total_Returns_Holdout                         193350
Total_Usage_Targeted                  7234021.993056
Total_Usage_Holdout                    458021.487778
Total_Repertoire_Targeted                    8493831
Total_Repertoire_Holdout                      531550
Total_Viewing_Sessions_Targeted              6090173
Total_Viewing_Sessions_Holdout                380977
Winback_Denom_Targeted                       2750122
Winback_Num_Targeted                           11284
Winback_Denom_Holdout                         229045
Winback_Num_Holdout                              891
Save_Denom_Targeted                          1650933
Save_Num_Targeted                              15292
Save_Denom_Holdout                            110750
Save_Num_Holdout                                 935
Upgrades_Denom_Targeted                     15

In [118]:
(df.sum().drop(['Report_Month', 'Account_Type']).squeeze() - df2.T.drop(['Report_Month']).squeeze()).sort_values()

Total_Paid_Churn_Num_Holdout                       67
EOM_Paid_Churn_Num_Holdout                        127
Total_New_Upgrade_Denom_Holdout                   822
Distinct_Cohort_Size_Holdout                     1035
EOM_Paid_Churn_Denom_Holdout                     1310
Total_Paid_Churn_Num_Targeted                    1421
Total_New_Upgrade_Num_Holdout                    1877
Total_Paid_Churn_Denom_Holdout                   2134
Total_Paid_Winbacks_Num_Holdout                  2189
Total_Paid_Winbacks_Denom_Holdout                4434
Upgrades_Num_Holdout                             4657
Upgrades_Denom_Holdout                           5256
Winback_Num_Holdout                             20325
Save_Num_Holdout                                25740
Winback_Denom_Holdout                           33478
Total_Paid_Winbacks_Num_Targeted                35207
Total_New_Upgrade_Num_Targeted                  35800
Total_Paid_Churn_Denom_Targeted                 62484
Save_Denom_Holdout          

In [26]:
%%bigquery df--project nbcu-ds-sandbox-a-001
SELECT distinct campaign_name
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
where lower(campaign_name) like '%trans%'

Query is running:   0%|          |

Downloading:   0%|          |

In [3]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT distinct campaign_name
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.SILVER_MPARTICLE_BRAZE`
where lower(campaign_name) like 'transactional%'

Query is running:   0%|          |

Downloading:   0%|          |

,campaign_name
0,TransactionalTermsOfUseEngaged20221028
1,Transactional_ProductAnnualRenewal_20220805
2,Transactional_OSDeprecation_20210218
3,TransactionalTermsOfUseNONEngaged20221028
4,TransactionalFamilyControlProfile20221209


In [12]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT * 
FROM `nbcu-ds-prod-001.PeacockDataMartSilver.INFORMATION_SCHEMA.COLUMNS`
where table_name = 'SILVER_MPARTICLE_BRAZE'

Query is running:   0%|          |

Downloading:   0%|          |

,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_generated,generation_expression,is_stored,is_hidden,is_updatable,is_system_defined,is_partitioning_column,clustering_ordinal_position,collation_name,column_default,rounding_mode
0,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,household_id,1,YES,STRING,NEVER,None,None,NO,None,NO,NO,1,NULL,NULL,None
1,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,adobe_tracking_id,2,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
2,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_name,3,YES,STRING,NEVER,None,None,NO,None,NO,NO,3,NULL,NULL,None
3,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_variant_name,4,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
4,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,campaign_tags,5,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
5,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,message_variant_channel,6,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
6,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_name,7,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
7,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_variant_name,8,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
8,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_step_name,9,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None
9,nbcu-ds-prod-001,PeacockDataMartSilver,SILVER_MPARTICLE_BRAZE,canvas_tags,10,YES,STRING,NEVER,None,None,NO,None,NO,NO,<NA>,NULL,NULL,None


# Email Engager Discrepancy

### Jan

In [38]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  COUNT(DISTINCT identity)
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView`
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-01-31', INTERVAL 1 YEAR) AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50126984


### Feb

In [39]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT  COUNT(DISTINCT identity)
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView`
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-02-28', INTERVAL 1 YEAR) AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,38570038


## All Opens

In [36]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity)
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE eventName = 'Email Opens'
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-01-31', INTERVAL 1 YEAR) AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,60164590


In [37]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity) 
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND date(eventTimestamp) BETWEEN DATE_SUB('2023-02-28', INTERVAL 1 YEAR) AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,59949647


## Hard Dates

In [42]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity)
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN '2022-01-31' AND '2023-01-31'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50126984


In [43]:
%%bigquery --project nbcu-ds-sandbox-a-001
SELECT COUNT(DISTINCT identity) 
FROM  `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` 
WHERE (eventName = 'Email Opens')
AND (machineOpen IS NULL)
AND date(eventTimestamp) BETWEEN '2022-02-28' AND '2023-02-28'
AND lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,38570038


### Email Acitivity History

In [33]:
%%bigquery --project nbcu-ds-sandbox-a-001

CREATE OR REPLACE TABLE `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Activity_Summary` AS
SELECT  DATE_TRUNC(DATE(eventTimestamp),MONTH)                                                                          AS month
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') THEN identity else null end)                               AS unique_all_email_opens
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN identity else null end)     AS unique_organic_email_opens
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN bid else null end)          AS unique_organic_email_opens_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is not null) THEN identity else null end) AS unique_machine_opens
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is not null) THEN bid else null end)      AS unique_machine_opens_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Sends') THEN identity else null end)                               AS unique_sends
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Sends') THEN bid else null end)                                    AS unique_sends_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Deliveries') THEN identity else null end)                          AS unique_deliveries
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Deliveries') THEN bid else null end)                               AS unique_deliveries_adobe_id
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Bounces') THEN identity else null end)                             AS unique_bounces
--       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Bounces') THEN bid else null end)                                  AS unique_bounces_adobe_id

       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') THEN identity else null end) OVER(order by DATE_TRUNC(DATE(eventTimestamp),MONTH) rows between 12 preceding and 1 preceding)                             AS unique_all_email_opens_1y_engagement
       ,COUNT(DISTINCT CASE WHEN (eventName = 'Email Opens') AND (machineOpen is null) THEN identity else null end) OVER(order by DATE_TRUNC(DATE(eventTimestamp),MONTH) rows between 12 preceding and 1 preceding)   AS unique_organic_email_opens_1y_engagement
FROM `nbcu-sdp-prod-003.sdp_persistent_views.BrazeMarketingView` a
--LEFT JOIN `nbcu-ds-sandbox-a-001.SLi_sandbox.Braze_Id_Adobe_Id_Map` b ON b.bid = a.identity
WHERE lower(campaignName) NOT LIKE 'transactional%' -- Exclude transactional emails
GROUP BY  1

Executing query with job ID: 5f729283-a1a6-4a91-8bfe-7820870d7474
Query executing: 63.01s

KeyboardInterrupt: 

In [40]:
%%bigquery --project nbcu-ds-sandbox-a-001
select * from `nbcu-ds-sandbox-a-001.SLi_sandbox.Email_Activity_Summary` order by month

Query is running:   0%|          |

Downloading:   0%|          |

,month,unique_organic_email_opens,unique_organic_email_opens_adobe_id,unique_machine_opens,unique_machine_opens_adobe_id,unique_sends,unique_sends_adobe_id,unique_deliveries,unique_deliveries_adobe_id,unique_bounces,unique_bounces_adobe_id
0,2020-07-01,0,0,0,0,0,0,0,0,0,0
1,2020-08-01,0,0,0,0,0,0,0,0,0,0
2,2020-09-01,3008622,3008622,0,0,12413715,12389742,12297056,12297056,2459,2459
3,2020-10-01,6416221,6416221,0,0,15521401,15480672,15355637,15355637,9829,9829
4,2020-11-01,7196354,7196354,0,0,19364472,19364472,19193302,19193302,5102,5102
5,2020-12-01,7716933,7716933,0,0,22110967,22110967,21898634,21898634,112194,112194
6,2021-01-01,9096252,9096252,0,0,24905131,24905131,24655848,24655848,7135,7135
7,2021-02-01,7197234,7197234,0,0,26992006,26992006,26702965,26702965,10902,10902
8,2021-03-01,9873612,9873612,0,0,30372028,30372028,30033356,30033356,27695,27695
9,2021-04-01,9815643,9815643,0,0,32628622,32628622,32259690,32259690,10787,10787
